In [3]:
import bk.load
import bk.compute
import matplotlib.pyplot as plt

import scipy.signal
import numpy as np

import bk.signal
import bk.compute
import bk.plot

import neuroseries as nts

In [4]:
bk.load.current_session_linux('/mnt/electrophy/Billel/BK-Dataset/ePhys/','Rat95/Rat95-20220411')

Rat : 91 on day : 3
Working with session Rat95-20220411 @ /mnt/electrophy/Billel/BK-Dataset/ePhys/Rat95/Rat95-20220411


True

In [5]:
TTL = bk.load.digitalin(0)

In [8]:
np.arange(0,5,1/20000)

array([0.00000e+00, 5.00000e-05, 1.00000e-04, ..., 4.99985e+00,
       4.99990e+00, 4.99995e+00])

In [13]:
%matplotlib qt
t = np.arange(0,len(TTL)/20_000,1/20_000)
plt.plot(t,TTL)

In [36]:
bk.load.current_session_linux('/mnt/electrophy/Billel/BK-Dataset/','Rat51/Rat51-20211202')

Rat : 51 on day : 19
Working with session Rat51-20211202 @ /mnt/electrophy/Billel/BK-Dataset/Rat51/Rat51-20211202


True

In [2]:
bk.load.current_session_linux('/mnt/electrophy/Billel/BK-Dataset/','Rat51/Rat51-20211208')

Rat : 51 on day : 21
Working with session Rat51-20211208 @ /mnt/electrophy/Billel/BK-Dataset/Rat51/Rat51-20211208


True

In [3]:
data = bk.load.digitalin(4)
stim = bk.compute.tone_intervals(data)
data = data[::20_000]
lfp = bk.load.lfp(1)
acc = []
for i in range(32,35):
    acc.append(bk.load.lfp(i,memmap=True))
motion = np.linalg.norm(acc,axis = 0)
acc = bk.load.lfp(34)

# lfp_stim = lfp.restrict(stim)
# np.save('lfpstim.npy',lfp_stim)

/!\ memmap is not compatible with volt_step /!\ 
/!\ memmap is not compatible with volt_step /!\ 
/!\ memmap is not compatible with volt_step /!\ 


In [4]:
f, t, sxx = scipy.signal.spectrogram(
    lfp, 1250, nperseg=1250*5, noverlap=1250*0)

f_delta = f<4
f_theta = (f>7) * (f<10)

r =  np.mean(sxx[f_theta,:],0)/np.mean(sxx[f_delta,:],0)
t_r = np.arange(0,len(r))
r = nts.Tsd(t,r,time_units='s')
# r_sm = bk.compute.nts_smooth(r,10,10)
r_sm = r

In [5]:
%matplotlib qt
fig,ax = plt.subplots(3,1,True)
ax[0].pcolormesh(t,f,np.log(sxx),shading = 'auto',vmin = 1,vmax = 8,rasterized = True)



ax[0].set_ylim(0,60)
a = ax[0].twinx()
a.plot(r_sm.as_units('s'),'white')
a.set_ylim(-5,20)
plt.sca(ax[1])
ax[1].plot(lfp.as_units('s'))
bk.plot.intervals(stim)

ax[1].set_ylim(-750,750)

# ax[2].plot(acc.as_units('s'))
ax[2].plot(acc.as_units('s').index,motion,color = 'red')

plt.suptitle(bk.load.session)
plt.tight_layout()
# plt.plot(r,'white')

/tmp/ipykernel_1275230/3302408814.py:2: MatplotlibDeprecationWarning: Passing the sharex parameter of subplots() positionally is deprecated since Matplotlib 3.3; the parameter will become keyword-only two minor releases later.
  fig,ax = plt.subplots(3,1,True)


In [55]:
bite = []
for i,j,k in os.walk('/mnt/electrophy/Billel/BK-Dataset/Rat51/Rat51-20211208/'):
    if not j:
        for l in k:
            if l == 'amplifier.dat':
                bite.append(os.path.getsize(os.path.join(i,l)))

In [56]:
import numpy as np
np.sum(bite)

37863418880

In [54]:
37863418880 / (20_000*2*35)

27045.2992

In [59]:
37863418880/ (20_000*2*35)

27045.2992